In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dropout,Concatenate,Flatten, ReLU, AveragePooling2D,Add, GlobalAveragePooling2D, Dense, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


inception net module

In [3]:
def inception_module(x,filter):
  f1,f2,f3,f4=filter
  # 1*1 filter
  conv1=Conv2D(f1,1,padding='same',activation='relu')(x)
  #3*3
  conv2=Conv2D(f2,1,padding='same',activation='relu')(x)
  conv2=Conv2D(f2,3,padding='same',activation='relu')(conv2)
  # 5*5
  conv3=Conv2D(f3,1,padding='same',activation='relu')(x)
  conv3=Conv2D(f3,5,padding='same',activation='relu')(conv3)
  # maxpool
  maxpool=MaxPooling2D(3,1,padding='same')(x)
  conv4=Conv2D(f4,1,padding='same',activation='relu')(maxpool)
  # catcatination
  cancat = Concatenate()([conv1,conv2,conv3,conv4])
  return cancat


In [8]:
def build_inception_net(input_shape,num_classes):
  inputs=Input(shape=input_shape)
  #layers1
  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(inputs)
  x=BatchNormalization()(x)
  x = MaxPooling2D(3, strides=2, padding='same')(x)
  # layers2
  x = Conv2D(192, (3, 3), padding='same',activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
# inception module
  x = inception_module(x, [64, 128, 32, 32])
  x = inception_module(x, [128, 192, 96, 64])
  x= MaxPooling2D((3, 3), strides=2, padding='same')(x)
  # second module
  x = inception_module(x, [192, 208, 48, 64])
  x = inception_module(x, [160, 224, 64, 64])
  x = inception_module(x, [128, 256, 64, 64])
  x = AveragePooling2D((7, 7), strides=2, padding='same')(x)
  # final layers
  x = Flatten()(x)
  x = Dropout(0.4)(x)
  x = Dense(1024, activation='relu')(x)
  outputs = Dense(num_classes, activation='softmax')(x)
  model = Model(inputs, outputs)
  return model



In [10]:
model = build_inception_net((32, 32, 3), 10)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 32, 32, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 16, 16, 64)     │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 16, 16, 64)     │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 8, 8, 64)       │              0 │ batch_normalization[0… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 8, 8, 192)      │        110,784 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 8, 8, 192)      │            768 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 4, 4, 192)      │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 4, 4, 128)      │         24,704 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 4, 4, 32)       │          6,176 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 4, 4, 192)      │              0 │ max_pooling2d_1[0][0]  │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 4, 4, 64)       │         12,352 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 4, 4, 128)      │        147,584 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 4, 4, 32)       │         25,632 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 4, 4, 32)       │          6,176 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 4, 4, 256)      │              0 │ conv2d_2[0][0],        │
│                           │                        │                │ conv2d_4[0][0],        │
│                           │                        │                │ conv2d_6[0][0],        │
│                           │                        │                │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)    

 Total params: 4,030,922 (15.38 MB)

 Trainable params: 4,030,410 (15.37 MB)

 Non-trainable params: 512 (2.00 KB)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [12]:
(x_trian,y_train),(x_test,y_test)=cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [13]:
x_train = x_trian.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
#

In [14]:
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 569s 702ms/step - accuracy: 0.3184 - loss: 1.8122 - val_accuracy: 0.4811 - val_loss: 1.4268
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 556s 695ms/step - accuracy: 0.5474 - loss: 1.2632 - val_accuracy: 0.5138 - val_loss: 1.3720
Epoch 3/10
156/782 ━━━━━━━━━━━━━━━━━━━━ 6:57 667ms/step - accuracy: 0.6134 - loss: 1.0936

KeyboardInterrupt: 